# Prices Paid Data Prep

## Importing Libraries


In [1]:
# import libraries
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import warnings
import requests
import time
warnings.filterwarnings('ignore')

## Prepare Data


### Import


In [2]:
# import the data
house_data = pd.read_csv(
    'raw_data/prices_paid.csv')
# display the data set to check it has imported correctly
house_data.head()

,{109BBF80-1E51-4910-8E2C-B124E1117A7A},34950,1995-06-09 00:00,BA3 3AH,F,N,L,COOMBEND HOUSE,7,COOMBEND,RADSTOCK,RADSTOCK.1,WANSDYKE,AVON,A,A.1
0,{D5233D67-2975-426B-B98D-B124E39331D9},133000,1995-06-19 00:00,SW18 5AG,T,N,F,142,NaN,ASTONVILLE STREET,LONDON,LONDON,WANDSWORTH,GREATER LONDON,A,A
1,{718BD35C-25E1-431C-8AF7-B124E63ED4E1},83000,1995-08-21 00:00,IP11 7PU,D,N,F,3,NaN,GARFIELD ROAD,FELIXSTOWE,FELIXSTOWE,SUFFOLK COASTAL,SUFFOLK,A,A
2,{E127F626-6247-4D95-A392-B124F4C0A558},59000,1995-07-27 00:00,PE21 0SF,D,Y,F,BEAUCADRE,NaN,GAYSFIELD ROAD,FISHTOFT,BOSTON,BOSTON,LINCOLNSHIRE,A,A
3,{B10762A0-2C9C-4C82-A885-B1250B55FFF9},51000,1995-02-28 00:00,ME7 4DF,T,N,F,41,NaN,FRANKLIN ROAD,GILLINGHAM,GILLINGHAM,GILLINGHAM,KENT,A,A
4,{BBAB55E8-216F-4FFA-B9EC-B4AD71F833EB},37000,1995-06-02 00:00,TN1 2ET,F,N,L,48,TOP FLOOR FLAT,UPPER GROSVENOR ROAD,TUNBRIDGE WELLS,TUNBRIDGE WELLS,TUNBRIDGE WELLS,KENT,A,A


In [3]:
# add column names to house_data
column_names = ['transaction_id', 'price', 'transfer_date', 'postcode', 'property_type', 'is_old_or_new', 'property_tenure',
                'house_number_or_name', 'unit_number', 'street', 'locality', 'town', 'district', 'county', 'ppd_transaction_category', 'record_status_monthly_file_only']
house_data.columns = column_names

In [4]:
# print information about house data
house_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29212383 entries, 0 to 29212382
Data columns (total 16 columns):
 #   Column                           Non-Null Count     Dtype 
---  ------                           --------------     ----- 
 0   transaction_id                   29212383 non-null  object
 1   price                            29212383 non-null  int64 
 2   transfer_date                    29212383 non-null  object
 3   postcode                         29165098 non-null  object
 4   property_type                    29212383 non-null  object
 5   is_old_or_new                    29212383 non-null  object
 6   property_tenure                  29212383 non-null  object
 7   house_number_or_name             29208198 non-null  object
 8   unit_number                      3463419 non-null   object
 9   street                           28748147 non-null  object
 10  locality                         18547494 non-null  object
 11  town                             29212383 non-nu

In [5]:
# get dimensions of the dataset
house_data.shape

(29212383, 16)

In [6]:
# check for columns that have null values for 50% of the entries
house_data.isnull().sum()/len(house_data)

transaction_id                     0.000000
price                              0.000000
transfer_date                      0.000000
postcode                           0.001619
property_type                      0.000000
is_old_or_new                      0.000000
property_tenure                    0.000000
house_number_or_name               0.000143
unit_number                        0.881440
street                             0.015892
locality                           0.365081
town                               0.000000
district                           0.000000
county                             0.000000
ppd_transaction_category           0.000000
record_status_monthly_file_only    0.000000
dtype: float64

In [7]:
# convert price to float type
house_data['price'] = house_data['price'].astype(float)

In [8]:
# convert transfer date to datetime
house_data['transfer_date'] = pd.to_datetime(house_data['transfer_date'])
# assert statement making sure of conversion to datetime
assert house_data['transfer_date'].dtype == 'datetime64[ns]'

In [9]:
# make a list of redundant columns
redundant_columns = ['house_number_or_name', 'unit_number', 'locality',
                    'street', 'record_status_monthly_file_only']
# drop redundant columns
house_data = house_data.drop(redundant_columns, axis=1)
# Assert redundant columns have been dropped
assert len(house_data.columns) != len(column_names)

In [10]:
# Find duplicates
duplicates = house_data.duplicated(subset='transaction_id', keep=False)
# Drop complete duplicates from house_data
unique_house_data = house_data.drop_duplicates()
duplicated_sales = unique_house_data[duplicates == True]
# Assert duplicates are processed
assert duplicated_sales.shape[0] == 0

In [11]:
# print unique values in columns
unique_house_data['property_type'].unique()
# Create mappings and replace
property_type_mapping = {'T': 'Terraced', 'D': 'Detached', 'F': 'Flats/Maisonettes',
                        'S': 'Semi-Detached', 'O': 'Other'}
# replace old values and change data type to categorical
unique_house_data['property_type'] = unique_house_data['property_type'].replace(
    property_type_mapping).astype('category')
# print new unique values in columns
unique_house_data['property_type'].unique()

['Terraced', 'Detached', 'Flats/Maisonettes', 'Semi-Detached', 'Other']
Categories (5, object): ['Detached', 'Flats/Maisonettes', 'Other', 'Semi-Detached', 'Terraced']

In [12]:
# print unique values in columns
unique_house_data['is_old_or_new'].unique()
# create mapping tp replace
old_or_new_mapping = {'N':'Old', 'Y':'New'}
unique_house_data['is_old_or_new'] = unique_house_data['is_old_or_new'].replace(
    old_or_new_mapping).astype('category')
# print new unique values in columns
unique_house_data['is_old_or_new'].unique()

['Old', 'New']
Categories (2, object): ['New', 'Old']

In [13]:
# print unique values in columns
unique_house_data['property_tenure'].unique()
# Remove rows with 'U' in property_tenure
unique_house_data = unique_house_data[unique_house_data['property_tenure'] != 'U']
# create mapping tp replace
property_tenure_mapping = {'F': 'Freehold', 'L': 'Leasehold'}
unique_house_data['property_tenure'] = unique_house_data['property_tenure'].replace(
    property_tenure_mapping).astype('category')
unique_house_data['property_tenure'].unique()

['Freehold', 'Leasehold']
Categories (2, object): ['Freehold', 'Leasehold']

In [14]:
# change ppd_transaction_category to category data type
unique_house_data['ppd_transaction_category'] = unique_house_data['ppd_transaction_category'].astype(
    'category')

In [15]:
# filter data for freehold transactions in merseyside from 2013 to 2023
merseyside_house_data = unique_house_data[(unique_house_data['county'] == 'MERSEYSIDE') & (
    unique_house_data['transfer_date'].dt.year >= 2013) & (unique_house_data['transfer_date'].dt.year <= 2023)].copy()

In [16]:
# Sort data by teansaction date
merseyside_house_data = merseyside_house_data.sort_values('transfer_date', ascending=True)
merseyside_house_data.head()

,transaction_id,price,transfer_date,postcode,property_type,is_old_or_new,property_tenure,town,district,county,ppd_transaction_category
18251630,{CD1FD346-02E2-40B9-AD20-AF02A78999D1},113000.0,2013-01-02,L31 2HS,Semi-Detached,Old,Freehold,LIVERPOOL,SEFTON,MERSEYSIDE,A
18351985,{EF89E3A8-2BD1-4347-8B9B-F2CEEB2E62DC},75000.0,2013-01-02,CH43 5RF,Flats/Maisonettes,Old,Leasehold,PRENTON,WIRRAL,MERSEYSIDE,A
18512520,{554C7E6D-FB60-4BF3-AEF0-F18802D4C110},385000.0,2013-01-02,WA12 0JF,Detached,Old,Freehold,NEWTON-LE-WILLOWS,ST HELENS,MERSEYSIDE,A
18275462,{3A27DE8C-0D42-41CC-8501-7367F7E98993},115000.0,2013-01-02,L13 5UP,Semi-Detached,Old,Freehold,LIVERPOOL,LIVERPOOL,MERSEYSIDE,A
18627036,{B5756393-923E-437D-B3BC-D21F5BAB0189},110000.0,2013-01-02,L3 5XY,Flats/Maisonettes,Old,Leasehold,LIVERPOOL,LIVERPOOL,MERSEYSIDE,A


In [17]:
merseyside_house_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 247523 entries, 18251630 to 28794487
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   transaction_id            247523 non-null  object        
 1   price                     247523 non-null  float64       
 2   transfer_date             247523 non-null  datetime64[ns]
 3   postcode                  246787 non-null  object        
 4   property_type             247523 non-null  category      
 5   is_old_or_new             247523 non-null  category      
 6   property_tenure           247523 non-null  category      
 7   town                      247523 non-null  object        
 8   district                  247523 non-null  object        
 9   county                    247523 non-null  object        
 10  ppd_transaction_category  247523 non-null  category      
dtypes: category(4), datetime64[ns](1), float64(1), object(5)
memo

In [18]:
# calculate the number of null values in postcode colum
merseyside_house_data['postcode'].isnull().sum()

736

In [19]:
# exclude rows with null postcode values
merseyside_house_data = merseyside_house_data[~merseyside_house_data['postcode'].isnull(
)]

In [20]:
# convert capital case columns to title case
merseyside_house_data['town'] = merseyside_house_data['town'].str.title()
merseyside_house_data['district'] = merseyside_house_data['district'].str.title()
merseyside_house_data['county'] = merseyside_house_data['county'].str.title()

### Export

In [21]:
# Save the filtered dataset
merseyside_house_data.to_csv(
    'clean_data/clean_property_price_data.csv', index=False)

In [25]:
merseyside_house_data

,transaction_id,price,transfer_date,postcode,property_type,is_old_or_new,property_tenure,town,district,county,ppd_transaction_category
18251630,{CD1FD346-02E2-40B9-AD20-AF02A78999D1},113000.0,2013-01-02,L31 2HS,Semi-Detached,Old,Freehold,Liverpool,Sefton,Merseyside,A
18351985,{EF89E3A8-2BD1-4347-8B9B-F2CEEB2E62DC},75000.0,2013-01-02,CH43 5RF,Flats/Maisonettes,Old,Leasehold,Prenton,Wirral,Merseyside,A
18512520,{554C7E6D-FB60-4BF3-AEF0-F18802D4C110},385000.0,2013-01-02,WA12 0JF,Detached,Old,Freehold,Newton-Le-Willows,St Helens,Merseyside,A
18275462,{3A27DE8C-0D42-41CC-8501-7367F7E98993},115000.0,2013-01-02,L13 5UP,Semi-Detached,Old,Freehold,Liverpool,Liverpool,Merseyside,A
18627036,{B5756393-923E-437D-B3BC-D21F5BAB0189},110000.0,2013-01-02,L3 5XY,Flats/Maisonettes,Old,Leasehold,Liverpool,Liverpool,Merseyside,A
...,...,...,...,...,...,...,...,...,...,...,...
28647698,{1A0C5C63-7B9D-7CBE-E063-4804A8C06C96},108000.0,2023-12-22,L10 1LD,Flats/Maisonettes,Old,Leasehold,Liverpool,Liverpool,Merseyside,A
28668565,{1061746E-D956-3C34-E063-4804A8C0F9E7},230000.0,2023-12-26,L6 7UH,Semi-Detached,Old,Leasehold,Liverpool,Liverpool,Merseyside,B
28435484,{1061746E-3651-3C34-E063-4804A8C0F9E7},120000.0,2023-12-27,L5 2BP,Flats/Maisonettes,Old,Leasehold,Liverpool,Liverpool,Merseyside,A
28591706,{12A8BAB6-6541-2125-E063-4804A8C08CC1},190000.0,2023-12-30,L19 1RZ,Terraced,Old,Freehold,Liverpool,Liverpool,Merseyside,A


In [26]:
def bulk_lookup_postcodes(postcodes, batch_size=100):
    """
    Perform a bulk lookup for postcodes using the postcodes.io API.

    Parameters:
    postcodes (list): A list of postcodes to lookup.
    batch_size (int): The number of postcodes to include in each batch (max 100).

    Returns:
    dict: A dictionary mapping postcodes to their respective data, including latitude, longitude, and termination data if applicable.
    """
    url = "https://api.postcodes.io/postcodes"
    headers = {'Content-Type': 'application/json'}
    results = {}

    # Process the postcodes in batches
    for i in range(0, len(postcodes), batch_size):
        batch = postcodes[i:i+batch_size]
        data = {"postcodes": batch}

        response = requests.post(url, json=data, headers=headers)

        if response.status_code == 200:
            response_data = response.json()
            for result in response_data['result']:
                postcode = result['query']
                if result['result'] is not None:
                    results[postcode] = {
                        'longitude': result['result']['longitude'],
                        'latitude': result['result']['latitude'],
                        'is_terminated': False
                    }
                else:
                    # If the postcode is terminated, set to None and mark as terminated
                    results[postcode] = {
                        'longitude': None,
                        'latitude': None,
                        'is_terminated': True
                    }
        else:
            print(
                f"Failed to retrieve data for batch starting at {i}: {response.status_code}")
        time.sleep(0.1)  # To avoid rate limiting

    return results



In [27]:

def handle_terminated_postcodes(postcode_data):
    """
    Handle postcodes that are marked as terminated by checking the terminated postcode API.

    Parameters:
    postcode_data (dict): A dictionary of postcodes with their data, including termination status.

    Returns:
    dict: Updated dictionary with longitude and latitude for terminated postcodes if available.
    """
    terminated_postcodes = [postcode for postcode,
                            data in postcode_data.items() if data['is_terminated']]

    if not terminated_postcodes:
        return postcode_data  # No terminated postcodes to handle

    # Query terminated postcodes in bulk (max 100 at a time)
    url = "https://api.postcodes.io/terminated_postcodes"
    headers = {'Content-Type': 'application/json'}
    batch_size = 100

    for i in range(0, len(terminated_postcodes), batch_size):
        batch = terminated_postcodes[i:i+batch_size]
        data = {"postcodes": batch}

        response = requests.post(url, json=data, headers=headers)

        if response.status_code == 200:
            response_data = response.json()
            for result in response_data['result']:
                postcode = result['query']
                if result['result'] is not None:
                    postcode_data[postcode]['longitude'] = result['result']['longitude']
                    postcode_data[postcode]['latitude'] = result['result']['latitude']
                else:
                    # If still no data, keep as None
                    print(f"No data found for terminated postcode: {postcode}")
        else:
            print(
                f"Failed to retrieve data for terminated postcodes batch starting at {i}: {response.status_code}")
        time.sleep(0.1)  # To avoid rate limiting

    return postcode_data


In [28]:
def add_long_lat_columns(df, postcode_column='postcode'):
    """
    Add longitude and latitude columns to the DataFrame based on the postcode column using Bulk Lookup.

    Parameters:
    df (DataFrame): The input DataFrame with a postcode column.
    postcode_column (str): The name of the column containing postcodes. Default is 'postcode'.

    Returns:
    DataFrame: The DataFrame with added 'longitude' and 'latitude' columns.
    """
    postcodes = df[postcode_column].unique().tolist()
    postcode_data = bulk_lookup_postcodes(postcodes)
    postcode_data = handle_terminated_postcodes(postcode_data)

    # Map the longitude and latitude back to the original DataFrame
    df['longitude'] = df[postcode_column].map(
        lambda x: postcode_data[x]['longitude'])
    df['latitude'] = df[postcode_column].map(
        lambda x: postcode_data[x]['latitude'])

    return df


In [29]:

# Example usage with your prices paid DataFrame
# Assuming your DataFrame is named 'prices_paid_df'


prices_paid_df = add_long_lat_columns(merseyside_house_data)

# Display the first few rows to verify
print(prices_paid_df.head())

Failed to retrieve data for terminated postcodes batch starting at 0: 404
Failed to retrieve data for terminated postcodes batch starting at 100: 404
Failed to retrieve data for terminated postcodes batch starting at 200: 404
                                  transaction_id     price transfer_date  \
18251630  {CD1FD346-02E2-40B9-AD20-AF02A78999D1}  113000.0    2013-01-02   
18351985  {EF89E3A8-2BD1-4347-8B9B-F2CEEB2E62DC}   75000.0    2013-01-02   
18512520  {554C7E6D-FB60-4BF3-AEF0-F18802D4C110}  385000.0    2013-01-02   
18275462  {3A27DE8C-0D42-41CC-8501-7367F7E98993}  115000.0    2013-01-02   
18627036  {B5756393-923E-437D-B3BC-D21F5BAB0189}  110000.0    2013-01-02   

          postcode      property_type is_old_or_new property_tenure  \
18251630   L31 2HS      Semi-Detached           Old        Freehold   
18351985  CH43 5RF  Flats/Maisonettes           Old       Leasehold   
18512520  WA12 0JF           Detached           Old        Freehold   
18275462   L13 5UP      Semi-Deta

In [32]:
prices_paid_df

,transaction_id,price,transfer_date,postcode,property_type,is_old_or_new,property_tenure,town,district,county,ppd_transaction_category,longitude,latitude
18251630,{CD1FD346-02E2-40B9-AD20-AF02A78999D1},113000.0,2013-01-02,L31 2HS,Semi-Detached,Old,Freehold,Liverpool,Sefton,Merseyside,A,-2.946158,53.519373
18351985,{EF89E3A8-2BD1-4347-8B9B-F2CEEB2E62DC},75000.0,2013-01-02,CH43 5RF,Flats/Maisonettes,Old,Leasehold,Prenton,Wirral,Merseyside,A,-3.038801,53.384498
18512520,{554C7E6D-FB60-4BF3-AEF0-F18802D4C110},385000.0,2013-01-02,WA12 0JF,Detached,Old,Freehold,Newton-Le-Willows,St Helens,Merseyside,A,-2.635595,53.479038
18275462,{3A27DE8C-0D42-41CC-8501-7367F7E98993},115000.0,2013-01-02,L13 5UP,Semi-Detached,Old,Freehold,Liverpool,Liverpool,Merseyside,A,-2.904433,53.412073
18627036,{B5756393-923E-437D-B3BC-D21F5BAB0189},110000.0,2013-01-02,L3 5XY,Flats/Maisonettes,Old,Leasehold,Liverpool,Liverpool,Merseyside,A,-2.971740,53.406042
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28647698,{1A0C5C63-7B9D-7CBE-E063-4804A8C06C96},108000.0,2023-12-22,L10 1LD,Flats/Maisonettes,Old,Leasehold,Liverpool,Liverpool,Merseyside,A,-2.923958,53.470978
28668565,{1061746E-D956-3C34-E063-4804A8C0F9E7},230000.0,2023-12-26,L6 7UH,Semi-Detached,Old,Leasehold,Liverpool,Liverpool,Merseyside,B,-2.938316,53.422489
28435484,{1061746E-3651-3C34-E063-4804A8C0F9E7},120000.0,2023-12-27,L5 2BP,Flats/Maisonettes,Old,Leasehold,Liverpool,Liverpool,Merseyside,A,-2.988893,53.421829
28591706,{12A8BAB6-6541-2125-E063-4804A8C08CC1},190000.0,2023-12-30,L19 1RZ,Terraced,Old,Freehold,Liverpool,Liverpool,Merseyside,A,-2.903954,53.360391


In [31]:
prices_paid_df[['longitude', 'latitude']].isnull().sum()

longitude    845
latitude     845
dtype: int64

In [48]:
prices_paid_df[prices_paid_df[['postcode', 'longitude', 'latitude']].isnull() == True]

,transaction_id,price,transfer_date,postcode,property_type,is_old_or_new,property_tenure,town,district,county,ppd_transaction_category,longitude,latitude
18251630,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18351985,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18512520,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18275462,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18627036,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28647698,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28668565,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28435484,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28591706,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
len(prices_paid_df.loc[prices_paid_df[prices_paid_df['latitude'].isnull()
                                            ].index.to_list()]['postcode'].unique())

219

In [58]:
prices_paid_df.loc[18746175]

transaction_id              {FE63F958-862B-474D-B3FA-43D383641785}
price                                                     300000.0
transfer_date                                  2013-02-11 00:00:00
postcode                                                   PR8 1PR
property_type                                             Terraced
is_old_or_new                                                  Old
property_tenure                                           Freehold
town                                                     Southport
district                                                    Sefton
county                                                  Merseyside
ppd_transaction_category                                         A
longitude                                                      NaN
latitude                                                       NaN
Name: 18746175, dtype: object